In [1]:
# !pip install sqlite3
# !pip install time
# !pip install psutil
# !pip install os
# !pip install faker

In [18]:
import sqlite3
import time
import random
import psutil
import os
from faker import Faker
fake = Faker("pt_br")

In [20]:
nome_banco = "sqlite_database.bd"
conexao = sqlite3.connect(nome_banco)
print(f"Banco de dados '{nome_banco}' criado.")
cursor = conexao.cursor()
cursor.execute("DROP TABLE IF EXISTS TESTE;")
cursor.execute("DROP TABLE IF EXISTS results;")
cursor.executescript(
    """
    CREATE TABLE IF NOT EXISTS teste (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        valor INTEGER
    );
    CREATE TABLE IF NOT EXISTS results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        time real,
        memori real,
        cpu real,
        nr_registros integer
    );
"""
)
conexao.commit()
print("Tabelas criadas com sucesso.")

Banco de dados 'sqlite_database.bd' criado.


OperationalError: database is locked

In [14]:
# Função para monitorar memória e CPU
def monitorar_recursos():
    processo = psutil.Process(os.getpid())
    memoria = processo.memory_info().rss / (1024 * 1024)  # Memória em MB
    cpu = psutil.cpu_percent(interval=1)  # Uso de CPU em %
    return memoria, cpu


# Função para medir tempo de execução
def medir_tempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fim = time.time()
        tempo_total = fim - inicio
        memoria, cpu = monitorar_recursos()
        print(
            f"Tempo de execução: {tempo_total:.2f}s | Memória usada: {memoria:.2f}MB | CPU usada: {cpu}%"
        )
        return resultado, tempo_total, memoria, cpu

    return wrapper


def registrar_resultados(tempo_processamento, memoria, cpu, nr_registros, conexao):
    cursor_registro = conexao
    cursor_registro.execute(
        "insert into results (time, memori, cpu, nr_registros) values (?,?,?,?)",
        (tempo_processamento, memoria, cpu, nr_registros),
    )
    conexao.commit()


# Contar registros
def contar_registros(conexao):
    cursor = conexao.cursor()
    cursor.execute("SELECT COUNT(*) FROM teste;")
    resultado = cursor.fetchone()
    print(f"Total de registros: {resultado[0]}")

In [11]:
@medir_tempo
def inserir_dados(conexao, num_registros):
    cursor = conexao.cursor()
    print(f"Inserindo {num_registros} registros...")
    cursor.execute("BEGIN TRANSACTION;")
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        cursor.execute("INSERT INTO teste (nome, valor) VALUES (?, ?)", (nome, valor))
    conexao.commit()
    print(f"{num_registros} registros inseridos com sucesso.")

In [15]:
# Testando o script
_, tempo_total, memoria, cpu = inserir_dados(conexao, num_registros=100000)  # Altere o valor aqui para testar o limite
registrar_resultados(tempo_total, memoria, cpu, 100000, conexao)
print(tempo_total, memoria, cpu)

Inserindo 100000 registros...


OperationalError: cannot start a transaction within a transaction

In [7]:
# Testando o script
inserir_dados(conexao, num_registros=1000000)  # Altere o valor aqui para testar o limite

Inserindo 1000000 registros...
1000000 registros inseridos com sucesso.
Tempo de execução: 12.36s | Memória usada: 76.61MB | CPU usada: 10.4%


In [8]:
# Testando o script
inserir_dados(conexao, num_registros=10000000)  # Altere o valor aqui para testar o limite

Inserindo 10000000 registros...
10000000 registros inseridos com sucesso.
Tempo de execução: 115.73s | Memória usada: 76.65MB | CPU usada: 10.3%


In [9]:
contar_registros(conexao=conexao)

Total de registros: 11100000


In [10]:
monitorar_recursos()

(76.65625, 7.1)

In [18]:
inserir_dados(conexao, 100000)

ProgrammingError: Cannot operate on a closed database.

In [12]:
select_cursor = conexao.cursor()
select_cursor.execute(
    "SELECT COUNT(nome) as contagem_nome, sum(valor) as contagem_valor from teste where nome like 'F%'"
)
resultado = select_cursor.fetchall()
print(resultado)

[(131907, 65967971820)]


In [19]:
conexao.close()